In [1]:
import pandas as pd
from decouple import config
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta, time
import time 
from sqlalchemy import text
import oracledb
import sys

In [2]:
def create_oracle_connection():
    try:
        oracledb.init_oracle_client(lib_dir=r"C:\Users\kings\Downloads\Nueva carpeta\instantclient_23_4")
        connection = oracledb.connect(
            user="User_oper",
            password="TmLQL$Yq.1",
            dsn="10.56.1.76:1527/WNET"
        )
        print("Conexión exitosa.")
        return connection
    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"Error al conectar: {error.message}")
        return None

# Reemplazar fetch_data por una función de ejemplo
def fetch_data():
    try:
        conn = create_oracle_connection()
        if conn is None:
            raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
        query = f"""SELECT
                    --ras.REDASISCOD,
                    --ras.REDASISDES,
                    cas.CENASICOD,
                    --cas.CENASIDES,
                    aho.AREHOSCOD,
                    --aho.AREHOSDES,
                    sho.SERVHOSCOD,
                    --sho.SERVHOSDES,
                    --act.ACTCOD,
                    --act.ACTDES,
                    --ace.ACTESPCOD,
                    --ace.ACTESPNOM,
                    act.ACTCOD || ace.ACTESPCOD AS ACTESPCOD,
                    med.PERNACFEC AS MEDNACFEC,
                    med.PERSEXOCOD AS MEDSEXO,
                    goc.GRUPOCUPCOD,
                    --goc.GRUPOCUPNOM,
                    cot.CONDTRABCOD,
                    --cot.CONDTRABNOM,
                    pac.PERDOCIDENNUM,
                    pac.PERNACFEC AS PACNACFEC,
                    pac.PERSEXOCOD AS PACSEXO,
                    cit.CITAMBPROCONFEC,
                    cit.CITAMBCREFEC,
                    cit.CITAMBHORCIT,
                    tci.TIPOCITACOD,
                    --tci.TIPOCITANOM,
                    coc.CONDCITACOD,
                    --coc.CONDCITANOM,
                    moc.MODOTORCITACOD,
                    --moc.MODOTORCITANOM,
                    eci.ESTCITCOD
                    --eci.ESTCITNOM
                    FROM
                    CTCAM10 cit
                    LEFT OUTER JOIN
                        CMCAS10 cas
                        ON cit.CITAMBCENASICOD = cas.CENASICOD
                    LEFT OUTER JOIN
                        CMRAS10 ras
                        ON cas.REDASISCOD = ras.REDASISCOD
                    LEFT OUTER JOIN
                        CMAHO10 aho
                        ON cit.CITAMBAREHOSCOD = aho.AREHOSCOD
                    LEFT OUTER JOIN
                        CMSHO10 sho
                        ON cit.CITAMBSERVHOSCOD = sho.SERVHOSCOD
                    LEFT OUTER JOIN
                        CMACT10 act
                        ON cit.CITAMBACTCOD = act.ACTCOD
                    LEFT OUTER JOIN
                        CMACE10 ace
                        ON cit.CITAMBACTCOD = ace.ACTCOD
                        AND cit.CITAMBACTESPCOD = ace.ACTESPCOD
                    LEFT OUTER JOIN
                        CMPER10 med
                        ON cit.CITAMBPERASISDOCIDENNUM = med.PERDOCIDENNUM
                    LEFT OUTER JOIN
                        CMPRS10 prs
                        ON cit.CITAMBPERASISDOCIDENNUM = prs.PERASISDOCIDENNUM
                    LEFT OUTER JOIN
                        CBGOC10 goc
                        ON prs.GRUPOCUPCOD = goc.GRUPOCUPCOD
                    LEFT OUTER JOIN
                        CBCOT10 cot
                        ON prs.CONDTRABCOD = cot.CONDTRABCOD
                    LEFT OUTER JOIN
                        CBTCI10 tci
                        ON cit.TIPOCITACOD = tci.TIPOCITACOD
                    LEFT OUTER JOIN
                        CBCOC10 coc
                        ON cit.CONDCITACOD = coc.CONDCITACOD
                    LEFT OUTER JOIN
                        CBMOC10 moc
                        ON cit.MODOTORCITACOD = moc.MODOTORCITACOD
                    LEFT OUTER JOIN
                        CBECI10 eci
                        ON cit.ESTCITCOD = eci.ESTCITCOD
                    LEFT OUTER JOIN
                        CMPER10 pac
                        ON cit.CITAMBPACSECNUM = pac.PERSECNUM
                    WHERE
                        act.ACTCOD = '91'
                        AND cit.CITAMBPROCONFEC >= TO_DATE('2024-08-01', 'YYYY-MM-DD')
                        AND cit.CITAMBPROCONFEC < TO_DATE('2024-08-02', 'YYYY-MM-DD')
                        AND eci.ESTCITCOD IN ('4', '5')
                        AND ras.REDASISCOD IN ('05')
                """
        df = pd.read_sql(query, conn)
    except Exception as e:
        print(f"Error: {e}")
        df = pd.DataFrame()
    finally:
        conn.close()  # Cerrar la conexión al final
    return df

In [3]:
df= fetch_data()

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3716012378.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
df.columns=df.columns.str.upper()

In [6]:
df.columns

Index(['CENASICOD', 'AREHOSCOD', 'SERVHOSCOD', 'ACTESPCOD', 'MEDNACFEC',
       'MEDSEXO', 'GRUPOCUPCOD', 'CONDTRABCOD', 'PERDOCIDENNUM', 'PACNACFEC',
       'PACSEXO', 'CITAMBPROCONFEC', 'CITAMBCREFEC', 'CITAMBHORCIT',
       'TIPOCITACOD', 'CONDCITACOD', 'MODOTORCITACOD', 'ESTCITCOD'],
      dtype='object')

In [7]:
query_CENASICOD = 'SELECT CENASICOD FROM CMCAS10'
CENASICOD = pd.read_sql_query(query_CENASICOD, con=create_oracle_connection())
query_AREHOSCOD = 'SELECT AREHOSCOD FROM CMAHO10'
AREHOSCOD = pd.read_sql_query(query_AREHOSCOD, con=create_oracle_connection())
query_SERVHOSCOD = 'SELECT SERVHOSCOD FROM CMSHO10'
SERVHOSCOD = pd.read_sql_query(query_SERVHOSCOD, con=create_oracle_connection())
query_ACTESPCOD = "SELECT ACTCOD || ACTESPCOD AS ACTESPCOD FROM CMACE10 WHERE ACTCOD = '91'"
ACTESPCOD = pd.read_sql_query(query_ACTESPCOD, con=create_oracle_connection())
query_GRUPOCUPCOD = 'SELECT GRUPOCUPCOD FROM CBGOC10'
GRUPOCUPCOD = pd.read_sql_query(query_GRUPOCUPCOD, con=create_oracle_connection())
query_CONDTRABCOD = 'SELECT CONDTRABCOD FROM CBCOT10'
CONDTRABCOD = pd.read_sql_query(query_CONDTRABCOD, con=create_oracle_connection())
query_TIPOCITACOD = 'SELECT TIPOCITACOD FROM CBTCI10'
TIPOCITACOD = pd.read_sql_query(query_TIPOCITACOD, con=create_oracle_connection())
query_CONDCITACOD = 'SELECT CONDCITACOD FROM CBCOC10'
CONDCITACOD = pd.read_sql_query(query_CONDCITACOD, con=create_oracle_connection())
query_MODOTORCITACOD = 'SELECT MODOTORCITACOD FROM CBMOC10'
MODOTORCITACOD = pd.read_sql_query(query_MODOTORCITACOD, con=create_oracle_connection())


Conexión exitosa.
Conexión exitosa.
Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  CENASICOD = pd.read_sql_query(query_CENASICOD, con=create_oracle_connection())
C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AREHOSCOD = pd.read_sql_query(query_AREHOSCOD, con=create_oracle_connection())
C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SERVHOSCOD = pd.read_sql_query(quer

Conexión exitosa.
Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ACTESPCOD = pd.read_sql_query(query_ACTESPCOD, con=create_oracle_connection())
C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  GRUPOCUPCOD = pd.read_sql_query(query_GRUPOCUPCOD, con=create_oracle_connection())


Conexión exitosa.
Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  CONDTRABCOD = pd.read_sql_query(query_CONDTRABCOD, con=create_oracle_connection())
C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  TIPOCITACOD = pd.read_sql_query(query_TIPOCITACOD, con=create_oracle_connection())


Conexión exitosa.
Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  CONDCITACOD = pd.read_sql_query(query_CONDCITACOD, con=create_oracle_connection())
C:\Users\kings\AppData\Local\Temp\ipykernel_7576\3310902535.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MODOTORCITACOD = pd.read_sql_query(query_MODOTORCITACOD, con=create_oracle_connection())
